In [2]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
import ROOT
%jsroot on

In [3]:
def new_rechits_all(df_):
    
    rechit_x = []
    rechit_y = []
    rechit_z = []
    rechit_ene = []
    
    
    rec_lay = df_.comb_rechit_z_trimAhcal
    
    sec_list = [0, 54, 154, 50000]
    lay_z_list = [13.87749958, 64.61849976, 166.92]
    
    for sec_z in range(3):
    
        rec_x = df_.comb_rechit_x_trimAhcal[np.logical_and(rec_lay<sec_list[sec_z+1], rec_lay>sec_list[sec_z])]
        rec_y = df_.comb_rechit_y_trimAhcal[np.logical_and(rec_lay<sec_list[sec_z+1], rec_lay>sec_list[sec_z])]
        rec_z = df_.comb_rechit_z_trimAhcal[np.logical_and(rec_lay<sec_list[sec_z+1], rec_lay>sec_list[sec_z])]
        rec_ene = df_.rechitEn_trimAhcal[np.logical_and(rec_lay<sec_list[sec_z+1], rec_lay>sec_list[sec_z])]

        new_cell_idxs = []
        new_rechit_x = []
        new_rechit_y = []
        new_rechit_z = []
        new_rechit_ene = []

        for rec_i in range(rec_x.size):


                cell_idx = np.array([rec_x[rec_i], rec_y[rec_i]]).tolist()


                if cell_idx not in new_cell_idxs:

                    new_cell_idxs.append(cell_idx)

                    new_rechit_x.append(rec_x[rec_i])
                    new_rechit_y.append(rec_y[rec_i])
                    new_rechit_z.append(lay_z_list[sec_z])
                    new_rechit_ene.append(rec_ene[rec_i])

                else: new_rechit_ene[new_cell_idxs.index(cell_idx)] += rec_ene[rec_i]

        rechit_x += new_rechit_x
        rechit_y += new_rechit_y
        rechit_z += new_rechit_z
        rechit_ene += new_rechit_ene
        
        new_cell_idxs = []
        new_rechit_x = []
        new_rechit_y = []
        new_rechit_z = []
        new_rechit_ene = []
        
    
    return [np.array(rechit_x), np.array(rechit_y), np.array(rechit_z), np.array(rechit_ene)]

In [4]:
data = uproot.open("/eos/user/n/nkasarag/NSM_5M_TB_HGCAL_flat_pion_samples/skimmed_ntuple_sim_config22_pions_0to1M_combinedHgc_Ahc_v46.root")

lst = [ 'rechitEn_trimAhcal',
 'comb_rechit_x_trimAhcal',
 'comb_rechit_y_trimAhcal',
 'comb_rechit_z_trimAhcal',
      'trueBeamEnergy']

In [5]:
num_events = data["pion_variables_v1"].pandas.df("trueBeamEnergy", flatten=False).trueBeamEnergy.size
print(num_events)


#event_lst = [0, 100000, 200000, 300000, 400000, num_events]
event_lst = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, num_events]
event_lst

836658


[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 836658]

In [6]:
for i in range(9):
    
    time1 = time.time()
    
    df = data["pion_variables_v1"].pandas.df(lst, flatten=False, entrystart=event_lst[i], entrystop=event_lst[i+1])
    df.reset_index(inplace=True)
    
    rechits = df.apply(new_rechits_all, axis=1)
    new_df = pd.DataFrame(rechits.values.tolist(), columns=["reduced_x", "reduced_y", "reduced_z", "reduced_ene"])

    
    new_df["trueBeamEnergy"] = df.trueBeamEnergy
    
    
    df_to_save = new_df
    
    
    df_to_save.to_pickle("pkl_files/red_all/event_%sto%s_0to1M.pickle"%(str(event_lst[i]), str(event_lst[i+1])))
    
    print(i+1, "-> mins = ", (time.time()-time1)/60)


1 -> mins =  7.662431414922079
2 -> mins =  8.190013953049977
3 -> mins =  8.060208650430043
4 -> mins =  8.17973381280899
5 -> mins =  8.229057522614797
6 -> mins =  8.903923821449279
7 -> mins =  7.798522901535034
8 -> mins =  10.536964591344198


IndexError: list index out of range

In [6]:
pwd


'/eos/home-i00/n/nkasarag/SWAN_projects/HGCal_Particle_Identificatin_and_Energy_Reconstruction/Reducing_longitudinal_granularity'